In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist=keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [4]:
train_images=train_images/255.0
test_images=test_images/255.0

In [5]:
train_images[0].shape

(28, 28)

In [6]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [7]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [8]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

C:\Users\91916\AppData\Local\Temp\ipykernel_24692\556418634.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [9]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=3,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output\Mnist Fashion\oracle.json
INFO:tensorflow:Reloading Tuner from output\Mnist Fashion\tuner0.json


In [10]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [11]:
model=tuner_search.get_best_models(num_models=1)[0]

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        102464    
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 80)                2478160   
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
Total params: 2,582,074
Trainable params: 2,582,074
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 169s 100ms/step - loss: 0.1283 - accuracy: 0.9524 - val_loss: 0.2645 - val_accuracy: 0.9190
Epoch 5/10
1688/1688 [==============================] - 165s 98ms/step - loss: 0.0917 - accuracy: 0.9657 - val_loss: 0.2726 - val_accuracy: 0.9147
Epoch 6/10
1688/1688 [==============================] - 171s 101ms/step - loss: 0.0645 - accuracy: 0.9759 - val_loss: 0.3593 - val_accuracy: 0.9165
Epoch 7/10
1688/1688 [==============================] - 175s 103ms/step - loss: 0.0472 - accuracy: 0.9830 - val_loss: 0.4098 - val_accuracy: 0.9132
Epoch 8/10
1688/1688 [==============================] - 170s 101ms/step - loss: 0.0337 - accuracy: 0.9880 - val_loss: 0.4699 - val_accuracy: 0.9073
Epoch 9/10
1688/1688 [==============================] - 169s 100ms/step - loss: 0.0310 - accuracy: 0.9888 - val_loss: 0.4918 - val_accuracy: 0.9027
Epoch 10/10
1688/1688 [==============================] - 174s 103ms/step - loss: 0.0237 - accuracy: 0.9912 - val_